# Les 05

## Inleiding

Dit is de werkcollege-oefening bij Les 05 van het vak *Advanced Datamining* (BFVH4DMN2). Bestudeer eerst de syllabus behorende bij deze les. Op BlackBoard kun je naast dit iPython/Jupyter notebook een tweetal Python-bestanden vinden. Sla deze op in dezelfde folder als dit notebook. Het verdient aanbeveling om voor elke les een aparte folder aan te maken.

- **template.py** bevat een opzet voor een module met object-georiënteerde implementaties van neurale netwerk algoritmen. Het doel van deze oefening is om deze code aan te vullen en uit te werken tot een correct werkend model. Open dit bestand in een code-editor naar keuze en sla het op als **model.py**. Vergeet niet om tijdens het uitwerken van deze oefening je aanpassingen in de editor telkens op te slaan voordat je de code in dit notebook uitvoert!

- **data.py** bevat een aantal functies die helpen bij het genereren en het visualiseren van de gebruikte datasets. Deze functies hoeven maar één keer ingelezen te worden en hoef je niet te wijzigen om deze opdracht correct uit te kunnen voeren.

Laten we dus beginnen om deze functies te importeren, samen met wat initialisatie-code: plaats de cursor in de cel hieronder en druk op Shift+Enter.

In [ ]:
%matplotlib inline
from importlib import reload

import model, data

## Batch learning

We beginnen dit werkcollege met het *refactoren* van onze bestaande code. Dat wil zeggen, we gaan deze opschonen en herstructureren om daarmee een overzichtelijkere module te krijgen.

Naast `__init__()` en `__str__()` bevatten onze diverse lagen momenteel vier functies, te weten `predict()`, `loss()`, `train()` en `fit()`.

- De `predict()`, `loss()` en `train()` functies bouwen eigenlijk op elkaar voort en hebben nogal wat overlap. Ze functioneren alledrie recursief over de lagen heen en werken op één instance tegelijkertijd. Die lenen zich ervoor om te worden samengevoegd tot één functie. We zullen deze daarbij gelijk generaliseren om met meerdere instances om te kunnen gaan.

- De `fit()` functie daarentegen wordt alleen maar in de eerste laag gebruikt en kan een hele dataset verwerken. Deze leent zich ervoor om te worden ondergebracht in een gespecialiseerde child-class `InputLayer()`.

De gegeven opzet voor je model bevat wederom de parent-class `Layer()`. Dit keer is ook diens child-class `InputLayer()` voorgedefinieerd. Deze beide classes zijn reeds volledig geïmplementeerd en hoeven in principe niet te worden aangepast. De overige child-classes `FullLayer()`, `SoftmaxLayer()` en `LossLayer()` bevatten onvolledige code; deze dien je nog aan te vullen. Een belangrijke verandering in deze laatste drie is dat deze nu nog slechts één inhoudelijke functie kennen, genaamd `fit()`. Deze neemt de rol van de `predict()`, `loss()` en `train()` functies over en retourneert drie resultaten overeenkomend met [i] de voorspelling, [ii] de loss, en [iii] de gradiënt van de loss naar de invoer. Diens gedrag is afhankelijk van de argumenten `xs`, `ys` en `alpha` die worden meegegeven.

- Als de functie wordt aangeroepen als `fit(xs)` dan berekent de functie op grond van de attributen $x_i$ een voorspelling $\hat{y}_i$. Het argument `xs` bevat echter niet langer de attributen van één instance, maar een geneste lijst met de attributen van hele batch aan instances. De geretourneerde uitkomst wordt daarmee ook een geneste lijst met voorspellingen.

- Als de functie wordt aangeroepen als `fit(xs, ys)` dan berekent de functie, naast de voorspellingen, met behulp van de gegeven gewenste uitkomsten $y_i$ ook de bijbehorende loss $l$. Opnieuw doet ie dit voor een hele batch aan instances, dus de geretourneerde losses vormen een lijst.

- Als de functie wordt aangeroepen als `fit(xs, ys, alpha)` dan berekent de functie, naast de voorspellingen en losses, ook de gradiënten van de loss en past deze de modelparameters aan met behulp van de gegeven learning rate $\alpha$. Omdat er een hele batch aan instances wordt meegegeven is de resulterende gradiënt opnieuw een geneste lijst met afgeleiden $\frac{\partial l}{\partial x_i}$ voor elke instance.

De gegeven class `InputLayer()` heeft reeds een `fit()` functie. Deze krijgt de attributen `xs` en gewenste uitkomsten `ys` van een dataset binnen, tezamen met een waarde voor de learning rate, het gewenste aantal te draaien epochs, en een nieuwe parameter die de grootte van de mini-batches aangeeft. Deze functie loopt herhaaldelijk door alle instances heen, verdeelt deze steeds willekeurig in mini-batches ter grootte van `size`, en traint hiermee het model met gebruikmaking van de `fit()` functie van de volgende laag. Bestudeer deze code zodat je begrijpt wat deze doet.

We kunnen een neuraal netwerkje instantiëren met de code hieronder. Controleer dat deze werkt.

In [ ]:
my_network = model.InputLayer() + \
             model.FullLayer(2, 2, act_func=model.relu_act_func) + \
             model.SoftmaxLayer() + \
             model.LossLayer(loss_func=model.crossentropy_loss_func)
print(my_network)

We beginnen achteraan in het neurale netwerk met de `LossLayer()`. Implementeer hiervan de `fit()` functie. Je kan natuurlijk gebruik maken van je oude functies `predict()`, `loss()` en `train()`, maar pas je bestaande implementatie aan zodat deze nu een *lijst* met invoeren betreffende meerdere instances kan ontvangen. De code kan er ongeveer als volgt komen uit te zien:

```
def fit(self, xs, ys=None, alpha=None):
    predictions = ...
    if ys is None:
        losses = None
    else:
        losses = ...
    if alpha is None:
        gradients = None
    else:
        gradients = ...
    return predictions, losses, gradients
```

Controleer dat je code werkt en hieronder de gewenste uitkomsten geeft voor de gegeven voorbeelddata.

In [ ]:
reload(model)

xs = [[0.0, 0.0, 0.0], [1.0, 2.0, 3.0]]
ys = [[0.0, 0.0, 0.0], [4.0, 5.0, 6.0]]
my_losslayer = model.LossLayer()
p, l, g = my_losslayer.fit(xs, ys, 0.0)
print('prediction =', p)
print('loss       =', l)
print('gradient   =', g)

Vervolgens passen we de `SoftmaxLayer()` aan. Maak deze wederom in staat om meerdere instances te verwerken. Blijf ook rekening houden met under-/overflow, zoals in de vorige les. De `fit()` functie kan ongeveer als volgt worden opgezet:

```
def fit(self, xs, ys, alpha):
    y_hats = ...
    predictions, losses, next_gradients = self.next.fit(y_hats, ys, alpha)
    if alpha is None:
        gradients = None
    else:
        gradients = ...
    return predictions, losses, gradients
```

Met onderstaande code kun je je uitwerking testen.

In [ ]:
reload(model)

xs = [[0.0, 0.0, 0.0, 0.0], [-1.0, 4.0, 2.0, 0.0]]
ys = [[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]]
my_softmaxlayer = model.SoftmaxLayer()
my_softmaxlayer.append(model.LossLayer(loss_func=model.crossentropy_loss_func))
p, l, g = my_softmaxlayer.fit(xs, ys, 0.0)
print('prediction =', p)
print('loss       =', l)
print('gradient   =', g)

Tenslotte stellen we de `FullLayer()` in staat om met meerdere instances om te gaan. Hierbij dienen niet alleen de voorspellingen, losses en gradiënten te worden doorgegeven, maar ook de biases en gewichten te worden bijgewerkt volgens gradient descent. Hierbij mag het effect van alle individuele instances bij elkaar opgeteld worden. Het resultaat ziet er daarmee ongeveer als volgt uit:

```
def fit(self, xs, ys=None, alpha=None):
    pre_activations = ...
    post_activations = ...
    predictions, losses, post_gradients = self.next.fit(post_activations, ys, alpha)
    if alpha is None:
        gradients = None
    else:
        pre_gradients = ...
        gradients = ...
        ... # Update the biases and weights using all instances
    return predictions, losses, gradients
```

Als het goed is werkt daarna het complete model. Dit valt te testen door de eerder gebruikte dataset op basis van de Newton fractal te fitten.

[![Newton fractal](https://upload.wikimedia.org/wikipedia/commons/thumb/d/db/Julia_set_for_the_rational_function.png/240px-Julia_set_for_the_rational_function.png)](https://upload.wikimedia.org/wikipedia/commons/d/db/Julia_set_for_the_rational_function.png)

Hieronder wordt een iets complexere opzet gebruikt dan in de vorige les. Er worden 4096 trainingsinstances gebruikt. Het fitten gebeurt in twee fasen, eerst met een ietwat grotere $\alpha=0.01$ in mini-batches van 16 instances per keer om tot een ruwe oplossing te komen, en daarna met een kleinere $\alpha=0.001$ en mini-batches van 256 instances om deze verder te verfijnen. De resultaten worden geplot aan de hand van 512 nieuwe testinstances. Het neurale model bevat:

- een input layer;

- een eerste hidden layer met 16 relu-neuronen;

- een tweede hidden layer met 8 relu-neuronen;

- een derde hidden layer met 3 lineaire neuronen;

- een softmax output layer;

- en een cross-entropy loss-functie.

Doet dit model het goed op het oog? Hoeveel details in de verdeling van de data kan het fitten? Varieer de diverse instellingen van het model of van de trainingsprocedure om te zien wat de invloed op de oplossing is.

In [ ]:
reload(model)

x_train, y_train = data.multinomial(3, num=4096)
x_test,  y_test  = data.multinomial(3, num=512)
my_network = model.InputLayer() + \
             model.FullLayer(2, 16, act_func=model.relu_act_func) + \
             model.FullLayer(16, 8, act_func=model.relu_act_func) + \
             model.FullLayer(8, 3, act_func=model.identity_act_func) + \
             model.SoftmaxLayer() + \
             model.LossLayer(loss_func=model.crossentropy_loss_func)
my_network.fit(x_train, y_train, alpha=0.01, epochs=30, size=16)
my_network.fit(x_train, y_train, alpha=0.001, epochs=30, size=256)
data.scatter(x_test, y_test, model=my_network)

## Adaptive learning

Vervolgens voeren we adaptive moments estimation (*Adam*) in om de convergentie van stochastic gradient descent robuuster te maken. Dit algoritme staat o.a. vermeld in het tekstboek *Deep Learning* van Goodfellow e.a. (p.301), het tekstboek *Neural Networks and Deep Learning* van Aggarwal (p.140), evenals de webpagina [An overview of gradient descent optimization algorithms](http://ruder.io/optimizing-gradient-descent/index.html#adam) van Ruder (gebruikmakend van uiteenlopende notaties).

Samengevat bestaat *Adam* uit een aangepaste update regel voor de modelparameters die weliswaar gebruik maakt van de gradient van de cost zoals die berekend wordt door middel van back-propagation, maar een aantal hulpvariabelen bijhoudt om lopende gemiddelden te bepalen van de afgeleide en diens kwadraat. In formulevorm komt dit neer op de volgende achtereenvolgende stappen.

\begin{equation*}
G\leftarrow\beta_{1}G+\left(1-\beta_{1}\right)\frac{\partial J}{\partial w}
\end{equation*}

\begin{equation*}
Q\leftarrow\beta_{2}Q+\left(1-\beta_{2}\right)\left(\frac{\partial J}{\partial w}\right)^{2}
\end{equation*}

\begin{equation*}
G'=\frac{G}{1-\beta_{1}^{t}}
\end{equation*}

\begin{equation*}
Q'=\frac{Q}{1-\beta_{2}^{t}}
\end{equation*}

\begin{equation*}
w\leftarrow w-\frac{\alpha}{\sqrt{Q'+\epsilon}}G'
\end{equation*}

In pseudo-code kan dit worden geïnitialiseerd met onderstaande code.

```
alpha = 0.001
beta1 = 0.9
beta2 = 0.99
beta1_power = 1.0
beta2_power = 1.0
epsilon = 1e-8
G[parameter] = [0.0 for parameter in biases_and_weights]
Q[parameter] = [0.0 for parameter in biases_and_weights]
w[parameter] = [startvalue for parameter in biases_and_weights]
```

De update regel die op elke minibatch wordt toegepast kan bijvoorbeeld worden geïmplementeerd als volgt.

```
beta1_power *= beta1
beta2_power *= beta2
for parameter in biases_and_weights:
    costgradient = sum(lossgradients[instance][parameter] for instance in minibatch)
    G[parameter] = beta1 * G[parameter] + (1.0-beta1) * costgradient
    Q[parameter] = beta2 * Q[parameter] + (1.0-beta2) * costgradient**2
    G_unbiased = G[parameter] / (1.0-beta1_power)
    Q_unbiased = Q[parameter] / (1.0-beta2_power)
    w[parameter] -= alpha * G_unbiased / sqrt(Q_unbiased + epsilon)
```

De parameters $\beta_1$ en $\beta_2$ kunnen naast $\alpha$ worden gespecificeerd als onderdeel van de aanroep van de `fit()` functie. Kies hierbij in de `InputLayer()` voor geschikte default waarden. Bedenk zelf welke andere waarden als variabelen van de child-class `FullLayer()` moeten worden bewaard.

Test je implementatie met onderstaande code.

In [ ]:
reload(model)

my_network = model.InputLayer() + \
             model.FullLayer(2, 16, act_func=model.relu_act_func) + \
             model.FullLayer(16, 8, act_func=model.relu_act_func) + \
             model.FullLayer(8, 3, act_func=model.identity_act_func) + \
             model.SoftmaxLayer() + \
             model.LossLayer(loss_func=model.crossentropy_loss_func)
my_network.fit(x_train, y_train, alpha=0.01, beta1=0.9, beta2=0.99, epochs=50, size=64)
data.scatter(x_test, y_test, model=my_network)

Als je het gewicht $\beta_1$ naar nul laat naderen verliest de variabele $G$ zijn geheugen en reduceert de methode tot *RMSProp*. Hieronder wordt dit algoritme uitgeprobeerd.

In [ ]:
reload(model)

my_network = model.InputLayer() + \
             model.FullLayer(2, 16, act_func=model.relu_act_func) + \
             model.FullLayer(16, 8, act_func=model.relu_act_func) + \
             model.FullLayer(8, 3, act_func=model.identity_act_func) + \
             model.SoftmaxLayer() + \
             model.LossLayer(loss_func=model.crossentropy_loss_func)
my_network.fit(x_train, y_train, alpha=0.01, beta1=0.0, beta2=0.9, epochs=50, size=64)
data.scatter(x_test, y_test, model=my_network)

**Gefeliciteerd!** Je hebt nu een neuraal netwerk geïmplementeerd dat met adaptive learning op mini-batches getraind kan worden.

Experimenteer weer hoe het algoritme zich gedraagt. Bijvoorbeeld:

- Hoe hangen de resultaten af van de grootte van de minibatch, van online learning met `size=1` tot batch learning tot `size=num`?

- Hoe hangt de convergentie af van niet alleen de learning rate parameter $\alpha$, maar ook de frictie parameters $\beta_1$ en $\beta_2$?

- Werken modellen met meer of minder lagen en/of met meer of minder neuronen nu beter?